In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

#machine learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, 
    r2_score, mean_absolute_percentage_error
)
import xgboost as xgb
import lightgbm as lgb
import shap

import sys
sys.path.append('..')
import config

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Set random seed
np.random.seed(config.RANDOM_SEED)

print("Libraries imported successfully")

Configuration loaded successfully!
Project root: C:\Users\hp\Desktop\gaf\Ghana-Armed-Forces-Personnel-Deployment-and-Attrition-Risk-Modeling
Random seed: 42
Target sample size: 1000 personnel
Libraries imported successfully


# Load Dataset

In [15]:
#load dataset 
data_path = config.PROCESSED_DATA_DIR/config.FEATURES_ENGINEERED_FILE
df = pd.read_csv(data_path)

print(f"Data Shape: {df.shape}")
print("Readiness score statistics:")
print(df['readiness_score'].describe())

Data Shape: (1000, 75)
Readiness score statistics:
count    1000.000000
mean       71.153700
std         9.110339
min        42.400000
25%        64.900000
50%        71.300000
75%        77.525000
max        99.300000
Name: readiness_score, dtype: float64


# Data Preparation

In [23]:
leakage_features = [
    'unit_avg_readiness', 'relative_readiness'
]


In [24]:
#seperate features and target 
X = df.drop(['attrition_risk', 'readiness_score'] + leakage_features, axis=1)
y = df['readiness_score']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

#encode categorical features 
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
print(f"\nCategorical features: {len(categorical_features)}")

#one-hot encode
X_encoded = pd.get_dummies(X, columns=categorical_features, drop_first=True)
print(f"\nFeatures after encoding: {X_encoded.shape}")

Features shape: (1000, 71)
Target shape: (1000,)

Categorical features: 12

Features after encoding: (1000, 98)


In [25]:
#train/test split (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y,
    test_size=config.TEST_SIZE,
    random_state=config.RANDOM_SEED
)

print("Dataset splits:")
print(f" Training: {X_train.shape[0]} samples ({X_train.shape[0]/len(df):.1%})")
print(f" Test: {X_test.shape[0]} samples ({X_test.shape[0]/len(df):.1%})")

Dataset splits:
 Training: 800 samples (80.0%)
 Test: 200 samples (20.0%)


In [26]:
#feature scaling 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#store feature names 
feature_names = X_encoded.columns.tolist()

print(f"Features scaled")
print(f"Total Features: {len(feature_names)}")

Features scaled
Total Features: 98


# Model Training 

#### Baseline: Linear Regression 

In [29]:
print("Training Linear Regreassion....")

lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

#predictions 
y_pred_lr = lr_model.predict(X_test_scaled)

#metrics for evaluation 
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
r2_lr = r2_score(y_test, y_pred_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)
mape_lr = mean_absolute_percentage_error(y_test, y_pred_lr)

print("\nLinear Regression trained....")
print(f"RMSE {rmse_lr:.2f}")
print(f"R square: {r2_lr:.3f}")
print(f"MAE: {mae_lr:.2f}")
print(f"MAPE:{mape_lr:.2f}")


Training Linear Regreassion....

Linear Regression trained....
RMSE 5.45
R square: 0.656
MAE: 4.35
MAPE:0.06


### Random Forest Regressor 

In [34]:
print("Training Random Forest Regressor...")

rf_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    min_samples_split=10,
    random_state=config.RANDOM_SEED,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

#prediction
y_pred_rf = rf_model.predict(X_test)

#metrics 
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mape_rf = mean_absolute_percentage_error(y_test, y_pred_rf)

print("\nRandom Forest trained")
print(f"RMSE: {rmse_rf:.2f}")
print(f"R squared: {r2_rf:.3f}")
print(f"MAE: {mae_rf:.2f}")
print(f"MAPE: {mape_rf:.2f}%")

Training Random Forest Regressor...

Random Forest trained
RMSE: 6.04
R squared: 0.578
MAE: 4.86
MAPE: 0.07%


### XGBoost Regressor 